# 查找鱼漂

因为本地机器的 GPU 只能支持 **tensorflow 1.10** ,故先将 colab 上的 **tensorflow** 降级.

In [0]:
!pip install tensorflow-gpu==1.10.0
!wget https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!dpkg -i cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
!apt-key add /var/cuda-repo-9-0-local/7fa2af80.pub
!apt-get update
!apt-get install cuda=9.0.176-1

     |████████████████████████████████| 253.3MB 47kB/s 
     |████████████████████████████████| 3.3MB 23.4MB/s 
     |████████████████████████████████| 12.2MB 31.8MB/s 
     |████████████████████████████████| 573kB 44.8MB/s 
ERROR: tensorflow 1.15.0 has requirement numpy<2.0,>=1.16.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.10.0 which is incompatible.
ERROR: spacy 2.1.8 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: imgaug 0.2.9 has requirement numpy>=1.15.0, but you'll have numpy 1.14.5 which is incompatible.
ERROR: fastai 1.0.58 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: cvxpy 1.0.25 has requirement numpy>=1.15, but you'll have numpy 1.14.5 which is incompatible.
ERROR: blis 0.2.4

--2019-10-25 08:55:09--  https://developer.nvidia.com/compute/cuda/9.0/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64-deb
Resolving developer.nvidia.com (developer.nvidia.com)... 192.229.162.216
Connecting to developer.nvidia.com (developer.nvidia.com)|192.229.162.216|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?GlpMbfMrWBOwBMx9I0WKZkU7TBBvrevAJb_Prqp36ODHXIPia9sHjUM-gDcJXJz3uTEjiNtMLqKXLUuQNRdqg3RN-E7WXGqviKshmQNkCpA-voaur9-tWeTORP2rmvt0HoPd8yFWcte9G7yDDiMCPdGya7SYmS-6O4coDtecweAsuBmfwAZ1N7XbzwU7ZUgPn0W1Q_eWPczpvqOuuneM [following]
--2019-10-25 08:55:10--  https://developer.download.nvidia.com/compute/cuda/9.0/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-0-local_9.0.176-1_amd64.deb?GlpMbfMrWBOwBMx9I0WKZkU7TBBvrevAJb_Prqp36ODHXIPia9sHjUM-gDcJXJz3uTEjiNtMLqKXLUuQNRdqg3RN-E7WXGqviKshmQNkCpA-vo

### 下载训练数据

- label_y.npy: 鱼漂的坐标点
- train_x.npy: 游戏截图
- label_y_cv.npy: 坐标点的验证集
- train_x_cv.npy: 游戏截图的验证集

In [0]:
!wget -O label_y.npy https://www.dropbox.com/s/yxrwp7pyck00yx6/label_y.npy?dl=1
!wget -O train_x.npy https://www.dropbox.com/s/xk90jhdhgv2vfaw/train_x.npy?dl=1
!wget -O label_y_cv.npy https://www.dropbox.com/s/lvtwn7wbhxey1tl/label_y_cv.npy?dl=1
!wget -O train_x_cv.npy https://www.dropbox.com/s/wry0jzvlzukwi55/train_x_cv.npy?dl=1

--2019-10-25 08:51:34--  https://www.dropbox.com/s/yxrwp7pyck00yx6/label_y.npy?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/yxrwp7pyck00yx6/label_y.npy [following]
--2019-10-25 08:51:34--  https://www.dropbox.com/s/dl/yxrwp7pyck00yx6/label_y.npy
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca4b49a46528f3fe4f64e086c1c.dl.dropboxusercontent.com/cd/0/get/ArG5UZkuXb--7E8M023oOKxam_abVqqwJENaq89VVZHARDW199eS8FDQNrt8SBQr43jIN3DTQ8IwFH_xi8BWHgY4UgwiEiWivB7q6I7SF0pXYhYFPyDbXL8jnPhUpYHMAOI/file?dl=1# [following]
--2019-10-25 08:51:34--  https://uca4b49a46528f3fe4f64e086c1c.dl.dropboxusercontent.com/cd/0/get/ArG5UZkuXb--7E8M023oOKxam_abVqqwJENaq89VVZHARDW199eS8FDQNrt8SBQr43jIN3DTQ8IwFH_xi8BWHgY4UgwiEiWivB7q6I7SF0pXYhYFPyD

### 构建训练模型

In [0]:
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.optimizers import Adam
import numpy as np


train_x_path = './train_x.npy'
label_y_path = './label_y.npy'
train_x_cv_path = './train_x_cv.npy'
label_y_cv_path = './label_y_cv.npy'
model_path = './my_model.h5'

x_train = np.load(train_x_path)
y_train = np.load(label_y_path)
x_cv = np.load(train_x_cv_path)
y_cv = np.load(label_y_cv_path)

print(x_train.shape)
print(y_train.shape)
print(y_train[3:6])
print(x_cv.shape)
print(y_cv.shape)

input_img = tf.keras.layers.Input(shape=(x_train.shape[1], x_train.shape[2], x_train.shape[3]))

x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu')(input_img)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
pos = tf.keras.layers.Dense(2, name='pos')(x)

model = tf.keras.models.Model(inputs=input_img, outputs=pos)

model.compile(
    loss = 'mean_squared_error',
    optimizer=Adam(),
    metrics=['acc'])

model.summary()

history = model.fit(
    x=x_train,
    y=y_train,
    epochs=500,
    verbose=2,
    validation_data=(x_cv, y_cv)
)

model.save(model_path)

print(np.hstack((model.predict(x_train[:10]), y_train[:10])))
print(np.hstack((model.predict(x_cv), y_cv)))

(70, 360, 360, 3)
(70, 2)
[[ 43 -28]
 [ 65  58]
 [-78  48]]
(7, 360, 360, 3)
(7, 2)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 360, 360, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 358, 358, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 179, 179, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 177, 177, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 88, 88, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 86, 86, 64)        18496     
__________________________________________________________

### 结论

误差不超过 6 个像素,对于最小的鱼漂也可以兼顾.